In [1]:
import xarray as xr
from pfpostproc.func import *
from pfpostproc.attrs import *
import pfpostproc.calc as calc
from datetime import datetime,timedelta
xr.set_options(keep_attrs=True);

In [2]:
domain = xr.open_dataset('domain.nc',
                         drop_variables=['pressure',
                                         'saturation',
                                         'perm_x',
                                         'perm_y'],
                         ).squeeze(drop=True)
domain = domain.rename({
        'perm_z': 'perm',
        'DZ_Multiplier': 'dz_mult'
    })

In [3]:
domain['mask'] = domain.mask.where(domain.mask==0,1)
domain['mask'] = domain.mask.isel(z=-1)
domain = domain * domain.mask.where(domain.mask==1)

In [4]:
domain.attrs = domain_attrs
domain.mannings.attrs = mannings_attrs
domain.perm.attrs = perm_attrs
domain.porosity.attrs = porosity_attrs
domain.specific_storage.attrs = specific_storage_attrs
domain.slopex.attrs = slopex_attrs
domain.slopey.attrs = slopey_attrs
domain.dz_mult.attrs = dz_mult_attrs
domain.mask.attrs = mask_attrs

In [5]:
domain['x'] = np.arange(domain.nx) + 1
domain['y'] = np.arange(domain.ny) + 1
domain['z'] = np.arange(domain.nz) + 1

In [6]:
tcoords = np.arange(datetime.strptime('2005-10-01 00:00', "%Y-%m-%d %H:%M"),
                    datetime.strptime('2006-10-01 00:00', "%Y-%m-%d %H:%M"),
                    timedelta(hours=1)).astype(datetime)

In [7]:
dem = raster_arr('sfs_dem.tif')
domain['elev'] = xr.DataArray(dem,dims=['y','x']) * domain.mask
domain.elev.attrs = elev_attrs

In [8]:
wrf = xr.open_dataset('wrf.nc')
wrf = wrf * domain.mask.values
wrf['precip'] = wrf.precip * 3.6 * domain.dx * domain.dy

In [9]:
wrf.shortwave.attrs = shortwave_attrs
wrf.longwave.attrs = longwave_attrs
wrf.precip.attrs = precip_attrs
wrf.temp.attrs = temp_attrs
wrf.wind_ew.attrs = wind_ew_attrs
wrf.wind_ns.attrs = wind_ns_attrs
wrf.atm_press.attrs = atm_press_attrs
wrf.sp_humid.attrs = sp_humid_attrs
wrf.attrs = domain_attrs

In [10]:
wrf['t'] = tcoords
wrf['x'] = np.arange(domain.nx) + 1
wrf['y'] = np.arange(domain.ny) + 1
wrf['z'] = np.arange(domain.nz) + 1

In [11]:
ds = xr.merge([domain,wrf])

In [12]:
ds

<xarray.Dataset>
Dimensions:           (t: 8760, x: 64, y: 128, z: 25)
Coordinates:
  * x                 (x) int64 1 2 3 4 5 6 7 8 9 ... 56 57 58 59 60 61 62 63 64
  * y                 (y) int64 1 2 3 4 5 6 7 8 ... 122 123 124 125 126 127 128
  * z                 (z) int64 1 2 3 4 5 6 7 8 9 ... 17 18 19 20 21 22 23 24 25
  * t                 (t) datetime64[ns] 2005-10-01 ... 2006-09-30T23:00:00
Data variables:
    mannings          (y, x) float64 nan nan nan nan nan ... nan nan nan nan nan
    perm              (z, y, x) float64 nan nan nan nan nan ... nan nan nan nan
    porosity          (z, y, x) float64 nan nan nan nan nan ... nan nan nan nan
    specific_storage  (z, y, x) float64 nan nan nan nan nan ... nan nan nan nan
    slopex            (y, x) float64 nan nan nan nan nan ... nan nan nan nan nan
    slopey            (y, x) float64 nan nan nan nan nan ... nan nan nan nan nan
    dz_mult           (z, y, x) float64 nan nan nan nan nan ... nan nan nan nan
    mask              (y, x) float64 nan nan nan nan nan ... nan nan nan nan nan
    elev              (y, x) float64 nan nan nan nan nan ... nan nan nan nan nan
    shortwave         (t, y, x) float64 nan nan nan nan nan ... nan nan nan nan
    longwave          (t, y, x) float64 nan nan nan nan nan ... nan nan nan nan
    precip            (t, y, x) float64 nan nan nan nan nan ... nan nan nan nan
    temp              (t, y, x) float64 nan nan nan nan nan ... nan nan nan nan
    wind_ew           (t, y, x) float64 nan nan nan nan nan ... nan nan nan nan
    wind_ns           (t, y, x) float64 nan nan nan nan nan ... nan nan nan nan
    atm_press         (t, y, x) float64 nan nan nan nan nan ... nan nan nan nan
    sp_humid          (t, y, x) float64 nan nan nan nan nan ... nan nan nan nan

In [13]:
ds.to_netcdf('domain_new.nc')